In [1]:
import numpy as np
import pandas as pd

# Trade-Offs in Missing Data Conventions

There are a number of schemes that have been developed to indicate the presence of missing data in a table or DataFrame. Generally, they revolve around one of two strategies:
- using a mask that globally indicates missing values, or 
- choosing a sentinel value that indicates a missing entry.


In the masking approach, the mask might be an entirely separate Boolean array, or it may involve appropriation of one bit in the data representation to locally indicate the null status of a value.

In the sentinel approach, the sentinel value could be some data-specific convention, such as indicating a missing integer value with -9999 or some rare bit pattern, or it could be a more global convention, such as indicating a missing floating-point value with NaN (Not a Number), a special value which is part of the IEEE floating-point specification.

None of these approaches is without trade-offs: 

- use of a **separate mask array** requires allocation of an additional Boolean array, which adds overhead in both storage and computation.
- A **sentinel value** reduces the range of valid values that can be represented, and may require extra (often non-optimized) logic in CPU and GPU arithmetic. Common special values like NaN are not available for all data types.

# Missing Data in Pandas

The way in which Pandas handles missing values is **constrained by its reliance on the NumPy package**, which does **not have a built-in notion of NA values for non-floating-point** data types.

While R contains four basic data types, NumPy supports far more than this: for example, while R has a single integer type, NumPy supports fourteen basic integer types once you account for available precisions, signedness, and endianness of the encoding. Reserving a specific bit pattern in all available NumPy types would lead to an unwieldy amount of overhead in special-casing various operations for various types, likely even requiring a new fork of the NumPy package. Further, for the smaller data types (such as 8-bit integers), sacrificing a bit to use as a mask will significantly reduce the range of values it can represent.

**NumPy does have support for masked arrays** – that is, arrays that have a separate Boolean mask array attached for marking data as "good" or "bad." Pandas could have derived from this, but the overhead in both storage, computation, and code maintenance makes that an unattractive choice.

# The choice of pandas for missing data 

With these constraints in mind, **Pandas chose to use sentinels for missing data**, and further chose to use two already-existing Python null values: 
- the special floating-point **NaN value,** 
- **and the Python None object**.
This choice has some side effects, as we will see, but in practice ends up being a good compromise in most cases of interest.

# None: Pythonic missing data

The first sentinel value used by Pandas is **None, a Python singleton object** that is often used for missing data in Python code. Because it is a Python object, None cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data type 'object' (i.e., arrays of Python objects):

In [2]:
vals1 = np.array([1, 2, None, 3])
vals1

array([1, 2, None, 3], dtype=object)

In [3]:
vals3 = np.array([1, 2, 4, 3])
vals3.dtype

dtype('int64')

The **dtype=object** for vals1 means that the *best common type representation NumPy could infer for the contents of the array is that they are Python objects*.

While this kind of object array is useful for some purposes, any operations on the data will be done at the Python level, with much more overhead than the typically fast operations seen for arrays with native types:

In [4]:
for dtype in ['object','int']:
    print('dtype=',dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype= object
89.8 ms ± 654 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype= int
1.62 ms ± 17.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)



In [5]:
# vals1.sum()

# error: 
# TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

# NaN: Missing numerical data

In [6]:
vals2 = np.array([1, 2, 3, np.nan, 5])
vals2.dtype

dtype('float64')

this array supports fast operations pushed into compiled code.

You should be aware that NaN is a bit like a data virus–it infects any other object it touches. Regardless of the operation, the result of arithmetic with NaN will be another NaN:

In [7]:
1 + np.nan

nan

In [8]:
5 * np.nan

nan

In [9]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

NumPy does provide some special aggregations that will ignore these missing values:

In [10]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(11.0, 1.0, 5.0)

# NaN and None in Pandas

NaN and None both have their place, and Pandas is built to handle the two of them nearly interchangeably, converting between them where appropriate:



In [11]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

For **types that don't have an available sentinel value**, Pandas automatically **type-casts** when NA values are present. For example, if we set a value in an integer array to np.nan, it will automatically be upcast to a floating-point type to accommodate the NA:

In [12]:
x = pd.Series( np.arange(5), dtype=int )
x

0    0
1    1
2    2
3    3
4    4
dtype: int64

In [13]:
x[1] = None
x

0    0.0
1    NaN
2    2.0
3    3.0
4    4.0
dtype: float64

Notice that in addition to casting the integer array to floating point, Pandas automatically converts the None to a NaN value. (Be aware that there is a proposal to add a native integer NA to Pandas in the future; as of this writing, it has not been included).

# Operating on Null Values
As we have seen, Pandas treats None and NaN as essentially interchangeable for indicating missing or null values. To facilitate this convention, there are several useful methods for detecting, removing, and replacing null values in Pandas data structures. They are:

In [14]:
data = pd.Series([1, np.nan, 'hello', None])
data.dtype

dtype('O')

### detecting null values

In [15]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [16]:
data[ data.notnull() ]

0        1
2    hello
dtype: object

### dropping null values

In [17]:
data.dropna()

0        1
2    hello
dtype: object

In [18]:
data.dropna() == data[ data.notnull() ]

0    True
2    True
dtype: bool

For a DataFrame, there are more options. Consider the following DataFrame:

In [19]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]], index=['a','b','c'], columns=['C','O','L'],)
df

,C,O,L
a,1.0,NaN,2
b,2.0,3.0,5
c,NaN,4.0,6


**We cannot drop single values from a DataFrame**; we can only drop full rows or full columns. Depending on the application, you might want one or the other, so dropna() gives a number of options for a DataFrame.

In [20]:
df.dropna()

,C,O,L
b,2.0,3.0,5


In [21]:
df.dropna( axis=1 ) == df.dropna( axis='columns')

,L
a,True
b,True
c,True


But this drops some good data as well; 

you might rather be interested in **dropping rows or columns with all NA values**, or a majority of NA values.

This can be specified through the **how or thresh parameters**, which allow *fine control* of the number of nulls to allow through. The default is how='any', such that any row or column (depending on the axis keyword) containing a null value will be dropped. You can also specify how='all', which will only drop rows/columns that are all null values:



In [22]:
# add one column of NaN's
df[3] = np.nan
df

,C,O,L,3
a,1.0,NaN,2,NaN
b,2.0,3.0,5,NaN
c,NaN,4.0,6,NaN


In [23]:
df.dropna(axis='columns', how='all')

,C,O,L
a,1.0,NaN,2
b,2.0,3.0,5
c,NaN,4.0,6


In [24]:
df.dropna(axis='rows', how='all')

,C,O,L,3
a,1.0,NaN,2,NaN
b,2.0,3.0,5,NaN
c,NaN,4.0,6,NaN


In [26]:
# For finer-grained control, the thresh parameter lets you specify 
# a minimum number of NON NULL values for the row/column to be kept:
df.dropna(axis='rows', thresh=3)

,C,O,L,3
b,2.0,3.0,5,NaN


In [30]:
df.dropna(axis=0, how='any')

,C,O,L,3


In [31]:
df.dropna(axis=1, how='any') 

,L
a,2
b,5
c,6


## Filling null values

Sometimes rather than dropping NA values, you'd rather replace them with a valid value.

This value might be a single number like zero, or it might be some sort of imputation or interpolation from the good values. 

You could do this in-place using the **isnull() method as a mask**, but because it is such a common operation Pandas provides the **fillna()** method, which returns a copy of the array with the null values replaced.

In [38]:
data = pd.Series([1, np.nan, 2, None, 3], index = list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [39]:
data.fillna(0, inplace=False)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [40]:
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [41]:
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

For DataFrames, the options are similar, but we can also specify an axis along which the fills take place:

In [42]:
df

,C,O,L,3
a,1.0,NaN,2,NaN
b,2.0,3.0,5,NaN
c,NaN,4.0,6,NaN


In [44]:
df.fillna(method='ffill', axis=1)

,C,O,L,3
a,1.0,1.0,2.0,2.0
b,2.0,3.0,5.0,5.0
c,NaN,4.0,6.0,6.0


In [45]:
df.fillna(method='bfill', axis=0)

,C,O,L,3
a,1.0,3.0,2,NaN
b,2.0,3.0,5,NaN
c,NaN,4.0,6,NaN


Notice that if a previous value is not available during a forward fill, the NA value remains.